
# 環境と流通の変化

:label: `sec_environment-and-distribution-shift`

前のセクションでは、モデルをさまざまなデータセットに適合させる、機械学習の実践的なアプリケーションを数多く取り上げてきました。それでも、私たちは、そもそもデータがどこから来たのか、モデルからの出力を最終的にどうするつもりなのかを考えるために立ち止まることはありませんでした。データを所有している機械学習開発者は、これらの基本的な問題を検討するために立ち止まることなく、モデルの開発を急いでしまうことがよくあります。

失敗した機械学習のデプロイメントの多くは、このパターンに遡ることができます。モデルは、テスト セットの精度で測定すると素晴らしいパフォーマンスを示しているように見えても、データの分布が突然変化すると、展開時に壊滅的に失敗することがあります。さらに巧妙なことに、モデルの展開そのものが、データの分布を混乱させる触媒となる場合があります。たとえば、誰が返済するのか、ローンの不履行を予測するモデルをトレーニングしたところ、申込者の靴の選択が不履行のリスクと関連していることが判明したとします (オックスフォードは返済を示し、スニーカーは不履行を示します)。その後、オックスフォードを履いているすべての申請者には融資を許可し、スニーカーを履いているすべての申請者には融資を拒否する傾向があるかもしれません。

この場合、私たちがパターン認識から意思決定に軽率に飛躍したことと、環境を批判的に考慮しなかったことにより、悲惨な結果が生じる可能性があります。まず、私たちが靴に基づいて意思決定をし始めるとすぐに、顧客はそれに気づき、行動を変えるようになりました。やがて、信用度が同時に向上することもなく、応募者全員がオックスフォードを着るようになります。機械学習の多くのアプリケーションには同様の問題がたくさんあるため、この点を少し理解してください。モデルベースの決定を環境に導入すると、モデルが壊れる可能性があります。

これらのトピックを 1 つのセクションで完全に扱うことはできませんが、ここでは、いくつかの一般的な懸念事項を明らかにし、これらの状況を早期に検出し、被害を軽減し、機械学習を責任を持って使用するために必要な批判的思考を刺激することを目的としています。解決策の中には、単純なもの (「正しい」データを求める) もあれば、技術的に難しいもの (強化学習システムの実装) もあれば、統計的予測の領域から完全に外に出て、倫理的問題に関する難しい哲学的問題に取り組む必要があるものもあります。アルゴリズムの応用。

## 流通シフトの種類

まず、データ分布が変化する可能性があるさまざまな方法と、モデルのパフォーマンスを回復するために何ができるかを考慮して、受動的な予測設定に固執します。 1 つの古典的な設定では、トレーニング データはある分布 $p_S(\mathbf{x},y)$ からサンプリングされたと仮定しますが、テスト データは別の分布 $p_T(\mathbf{x) から抽出されたラベルのないサンプルで構成されていると仮定します},y)$。すでに私たちは厳粛な現実に直面しなければなりません。 $p_S$ と $p_T$ が互いにどのように関係するかについての仮定がなければ、堅牢な分類器を学習することは不可能です。

犬と猫を区別したいという二項分類問題を考えてみましょう。分布が任意の方法でシフトする可能性がある場合、私たちの設定では、入力全体の分布が一定のままである病的なケースが許可されます: $p_S(\mathbf{x}) = p_T(\mathbf{x})$ しかし、ラベルはすべて反転: $p_S(y \mid \mathbf{x}) = 1 - p_T(y \mid \mathbf{x})$。言い換えれば、もし神が将来すべての「猫」が犬になり、以前は「犬」と呼ばれていたものも猫になると突然決定できたとしたら、入力 $p(\mathbf{x} )$ の場合、この設定と分布がまったく変化しなかった設定を区別することはできません。

幸いなことに、データが将来どのように変化するかについてのいくつかの制限された仮定の下では、原則に基づいたアルゴリズムがシフトを検出し、場合によってはその場で適応することができ、元の分類器の精度が向上します。

### 共変量シフト

分布シフトのカテゴリの中で、共変量シフトが最も広く研究されている可能性があります。ここで、入力の分布は時間の経過とともに変化する可能性があるが、ラベリング関数、つまり条件付き分布 $P(y \mid \mathbf{x})$ は変化しないと仮定します。共変量 (特徴) の分布のシフトによって問題が発生するため、統計学者はこの問題を*共変量シフト*と呼びます。因果関係を援用せずに分布シフトについて推論できる場合もありますが、$\mathbf{x}$ が $y$ を引き起こすと考えられる設定では共変量シフトを呼び出すのが自然な仮定であることに注意してください。

猫と犬を区別するという課題を考えてみましょう。トレーニング データは、 :numref: `fig_cat-dog-train`のような画像で構成されている可能性があります。 

![](../img/cat-dog-train.svg) :label: `fig_cat-dog-train`

テスト時には、 :numref: `fig_cat-dog-test`で画像を分類するように求められます。 

![](../img/cat-dog-test.svg) :label: `fig_cat-dog-test`

トレーニング セットは写真で構成されていますが、テスト セットには漫画のみが含まれています。テスト セットとは大幅に異なる特性を持つデータセットでトレーニングを行う場合、新しい領域に適応する方法についての一貫した計画がなければ、問題が発生する可能性があります。

### ラベルシフト

*ラベルシフトは*逆の問題を説明します。ここでは、ラベル周辺 $P(y)$ は変化する可能性があるが、クラス条件付き分布 $P(\mathbf{x} \mid y)$ はドメイン全体で固定されたままであると仮定します。ラベル シフトは、$y$ が $\mathbf{x}$ を引き起こすと考えられる場合に行う合理的な仮定です。たとえば、診断の相対的な有病率が時間の経過とともに変化する場合でも、症状 (またはその他の症状) を考慮して診断を予測したい場合があります。病気は症状を引き起こすため、ラベルシフトはここでは適切な仮定です。一部の縮退ケースでは、ラベル シフトと共変量シフトの仮定が同時に成立する可能性があります。たとえば、ラベルが決定的である場合、$y$ が $\mathbf{x}$ を引き起こす場合でも、共変量シフト仮定は満たされます。興味深いことに、このような場合、ラベル シフトの仮定から流れるメソッドを使用することが有利なことがよくあります。これは、これらのメソッドでは、深層学習では高次元である傾向がある入力のように見えるオブジェクトではなく、ラベルのように見えるオブジェクト (多くの場合低次元) を操作する傾向があるためです。



### コンセプトシフト

また、ラベルの定義自体が変更される可能性がある場合に発生する、*概念のシフト*に関連する問題に遭遇する可能性があります。奇妙に聞こえますが、*猫は**猫*ですよね？ただし、他のカテゴリは時間の経過とともに使用法が変更される可能性があります。精神疾患の診断基準、ファッショナブルなもの、および役職はすべて、かなりの概念の変化の対象となります。データのソースを地理別に移動しながら米国中を移動すると、 :numref: `fig_popvssoda`に示すように、*ソフトドリンク*の名前の分布に関してかなりの概念の変化が見つかることがわかります。 

![](../img/popvssoda.png) :幅: `400px` :ラベル: `fig_popvssoda`

機械翻訳システムを構築する場合、分布 $P(y \mid \mathbf{x})$ は場所によって異なる可能性があります。この問題は発見するのが難しい場合があります。私たちは、変化は時間的または地理的な意味で徐々にしか起こらないという知識を活用したいと考えるかもしれません。

## 流通シフトの例

形式主義とアルゴリズムを詳しく説明する前に、共変量や概念の変化が明らかではないいくつかの具体的な状況について説明します。

### 医療診断

がんを検出するアルゴリズムを設計したいと想像してください。健康な人と病気の人からデータを収集し、アルゴリズムをトレーニングします。問題なく機能し、高い精度が得られるため、医療診断分野で成功する準備ができていると判断できます。*そんなに早くない。*

トレーニング データを生成した分布と、実際に遭遇する分布は大きく異なる可能性があります。これは、私たち（著者）の何人かが何年も前に働いていた不幸なスタートアップで起こりました。彼らは主に高齢男性が罹患する病気の血液検査を開発しており、患者から採取した血液サンプルを使ってそれを研究したいと考えていた。しかし、健康な男性から血液サンプルを採取することは、すでにシステムに入っている病気の患者から採取することよりもかなり困難です。それを補うために、このスタートアップは、テストを開発する際の健康対照として役立つよう、大学キャンパス内の学生からの献血を募った。次に彼らは、病気を検出するための分類器の構築を私たちに手伝ってもらえないかと尋ねました。

私たちが彼らに説明したように、健康なコホートと病気のコホートをほぼ完璧な精度で区別するのは確かに簡単です。しかし、それは被験者の年齢、ホルモンレベル、身体活動、食事、アルコール摂取量、その他病気に関係のない多くの要素が異なっていたためです。これは実際の患者には当てはまらない可能性があります。サンプリング手順により、極端な共変量シフトが発生することが予想されます。さらに、このケースは従来の方法では修正できる可能性が低いです。要するに、彼らは多額のお金を無駄にしたのです。

### 自動運転車

ある企業が自動運転車の開発に機械学習を活用したいと考えているとします。ここでの重要なコンポーネントの 1 つは路側探知機です。注釈付きの実際のデータは入手するのに費用がかかるため、彼らはゲーム レンダリング エンジンからの合成データを追加のトレーニング データとして使用するという (賢明ではあるが疑わしい) アイデアを思いつきました。これは、レンダリング エンジンから描画された「テスト データ」では非常にうまく機能しました。ああ、実際の車の中では大惨事でした。結局のところ、道端は非常に単純なテクスチャでレンダリングされていました。さらに重要なことは、*すべての*道路脇が*同じ*テクスチャでレンダリングされており、道路脇の検出器がこの「特徴」を非常に迅速に学習したことです。

米陸軍が初めて森の中で戦車を探知しようとしたときに、同様のことが起こりました。彼らは戦車なしで森の航空写真を撮り、その後戦車を森に乗り入れてもう一度写真を撮りました。分類器は*完全に*機能しているように見えました。残念ながら、それは影のある木と影のない木を区別する方法を学習しただけでした。最初の写真セットは早朝に撮影され、2 番目のセットは正午に撮影されました。

### 非定常分布

分布がゆっくりと変化し (*非定常分布*とも呼ばれる)、モデルが適切に更新されない場合、さらに微妙な状況が発生します。以下に典型的なケースをいくつか示します。
- 私たちはコンピュテーショナル広告モデルをトレーニングした後、それを頻繁に更新できません（たとえば、iPad と呼ばれるあいまいな新しいデバイスが発売されたばかりであることを組み込むのを忘れます）。
- スパムフィルターを構築します。これまでに確認したすべてのスパムを検出するのにうまく機能します。しかしその後、スパマーは賢くなり、これまでに見たものとは異なる新しいメッセージを作成します。
- 製品の推奨システムを構築します。冬の間ずっと機能しますが、クリスマス後もずっとサンタ帽子を推奨し続けます。

### その他の逸話
- 顔検出器を構築します。すべてのベンチマークでうまく機能します。残念ながら、テスト データでは失敗します。問題となる例は、顔が画像全体を占めるクローズアップです (トレーニング セットにはそのようなデータはありませんでした)。
- 私たちは米国市場向けの Web 検索エンジンを構築しており、それを英国に展開したいと考えています。
- 私たちは、大規模なクラス セットのそれぞれがデータセット内で均等に表現される大規模なデータセット (たとえば、それぞれ 1000 枚の画像で表される 1000 カテゴリ) をコンパイルすることによって画像分類器をトレーニングします。次に、このシステムを現実世界に展開します。そこでは、写真の実際のラベル分布は明らかに不均一です。



## 分布ずれの修正

すでに説明したように、トレーニング分布とテスト分布 $P(\mathbf{x}, y)$ が異なる場合が多くあります。場合によっては、共変量、ラベル、概念の変化にもかかわらず、幸運なことにモデルが機能することがあります。また、原則に基づいた戦略を採用して変化に対処することで、より良い成果を上げることができる場合もあります。このセクションの残りの部分は、かなり技術的な内容になります。この内容は以降の概念の前提条件ではないため、せっかちな読者は次のセクションに進んでください。

### 経験的リスクとリスク

:label: `subsec_empirical-risk-and-risk`

まず、モデルのトレーニング中に正確に何が起こっているかを考えてみましょう。トレーニング データ ${(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)}$ の特徴と関連するラベルを繰り返し処理します。そして、ミニバッチごとにモデル $f$ のパラメーターを更新します。簡単にするために、正則化を考慮しないため、トレーニングの損失を大幅に最小限に抑えます。

 $$\mathop{\mathrm{minimize}} *f \frac{1}{n} \sum* {i=1}^nl(f(\mathbf{x}_i), y_i),$$ :eqlabel: `eq_empirical-risk-min`

ここで、$l$ は、予測 $f(\mathbf{x}_i)$ に関連付けられたラベル $y_i$ が「どのくらい悪いかが」を測定する損失関数です。統計学者は、この用語を :eqref: `eq_empirical-risk-min`*経験的リスク*と呼びます。*経験的リスクは、**リスクを*近似するためのトレーニング データ全体にわたる平均損失であり、これは、真の分布 $p(\mathbf{x},y)$ から抽出されたデータの母集団全体にわたる損失の期待値です。

 $$E_{p(\mathbf{x}, y)} [l(f(\mathbf{x}), y)] = \int\int l(f(\mathbf{x}), y) p( \mathbf{x}, y) ;d\mathbf{x}dy.$$ :eqlabel: `eq_true-risk`

ただし、実際には通常、データの母集団全体を取得することはできません。したがって、 :eqref: `eq_empirical-risk-min`で*経験的リスクを最小化する経験的リスクの最小化は*、リスクを最小限に近似することを目的とした機械学習の実用的な戦略です。

### 共変量シフト補正

:label: `subsec_covariate-shift-correction`

データ $(\mathbf{x}_i, y_i)$ にラベルを付けた依存関係 $P(y \mid \mathbf{x})$ を推定したいとします。残念ながら、観測値 $\mathbf{x}_i$ は、*ターゲット分布*$p(\mathbf{x})$ ではなく、*ソース分布*$q(\mathbf{x})$ から抽出されています。幸いなことに、依存関係の仮定は、条件付き分布が変化しないことを意味します: $p(y \mid \mathbf{x}) = q(y \mid \mathbf{x})$。ソース分布 $q(\mathbf{x})$ が「間違っている」場合、リスクに次の単純な恒等式を使用することでそれを修正できます。

 $$ \begin{aligned} \int\int l(f(\mathbf{x}), y) p(y \mid \mathbf{x})p(\mathbf{x}) ;d\mathbf{x} dy = \int\int l(f(\mathbf{x}), y) q(y \mid \mathbf{x})q(\mathbf{x})\frac{p(\mathbf{x})} {q(\mathbf{x})} ;d\mathbf{x}dy。 \end{整列} $$

言い換えれば、各データ例を、正しい分布から抽出された確率と間違った分布から抽出された確率の比によって再重み付けする必要があります。

 $$\beta_i \stackrel{\mathrm{def}}{=} \frac{p(\mathbf{x}_i)}{q(\mathbf{x}_i)}.$$

各データ例 $(\mathbf{x}_i, y_i)$ の重み $\beta_i$ を代入すると*、重み付けされた経験的リスク最小化*を使用してモデルをトレーニングできます。

 $$\mathop{\mathrm{minimize}} *f \frac{1}{n} \sum* {i=1}^n \beta_i l(f(\mathbf{x}_i), y_i).$$ :eqlabel : `eq_weighted-empirical-risk-min`

残念ながら、その比率はわかりません。そのため、何か役立つことを行う前に、比率を推定する必要があります。最小ノルムまたは最大エントロピー原理を使用して期待値演算子を直接再調整しようとする派手な演算子理論的アプローチなど、多くの方法が利用可能です。このようなアプローチでは、両方の分布から抽出されたサンプルが必要であることに注意してください。つまり、テスト データへのアクセスなどによって「真の」 $p$ と、トレーニング セット $q$ の生成に使用されるサンプルです (後者は自明です)利用可能）。ただし、必要なのは機能 $\mathbf{x} \sim p(\mathbf{x})$ のみであることに注意してください。ラベル $y \sim p(y)$ にアクセスする必要はありません。

この場合、元のアプローチとほぼ同じくらい良好な結果が得られる非常に効果的なアプローチが存在します。ロジスティック回帰は、バイナリ分類のソフトマックス回帰 (:numref: `sec_softmax`を参照) の特殊なケースです。推定確率比を計算するために必要なのはこれだけです。 $p(\mathbf{x})$ から抽出されたデータと $q(\mathbf{x})$ から抽出されたデータを区別するための分類器を学習します。 2 つのディストリビューションを区別できない場合は、関連するインスタンスが 2 つのディストリビューションのいずれかに由来する可能性が等しいことを意味します。一方、十分に区別できるインスタンスは、それに応じて大幅に過大または過少に重み付けされる必要があります。

簡単にするために、両方の分布 $p(\mathbf{x})$ と $q(\mathbf{x})$ からそれぞれ同じ数のインスタンスがあると仮定します。ここで、$p$ から引き出されたデータの場合は $1$、$q$ から引き出されたデータの場合は $-1$ である $z$ ラベルを示します。この場合、混合データセット内の確率は次のように求められます。

 $$P(z=1 \mid \mathbf{x}) = \frac{p(\mathbf{x})}{p(\mathbf{x})+q(\mathbf{x})} \text{したがって } \frac{P(z=1 \mid \mathbf{x})}{P(z=-1 \mid \mathbf{x})} = \frac{p(\mathbf{x})}{ q(\mathbf{x})}.$$

したがって、ロジスティック回帰アプローチを使用すると、 $P(z=1 \mid \mathbf{x})=\frac{1}{1+\exp(-h(\mathbf{x}))}$ ( $h$ はパラメータ化された関数です)、次のようになります。

 $$ \beta_i = \frac{1/(1 + \exp(-h(\mathbf{x}_i)))}{\exp(-h(\mathbf{x}_i))/(1 + \exp (-h(\mathbf{x}_i)))} = \exp(h(\mathbf{x}_i))。 $$

その結果、2 つの問題を解決する必要があります。1 つは両方の分布から抽出されたデータを区別する問題で、次に :eqref: `eq_weighted-empirical-risk-min`で重み付けされた経験的リスク最小化問題で、$\beta_i$ によって項を重み付けします。 。

これで、補正アルゴリズムを説明する準備が整いました。トレーニング セット ${(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)}$ とラベルのないテスト セット ${\mathbf{u}_1, \ があるとします。 ldots、\mathbf{u}_m}$。共変量シフトの場合、すべての $1 \leq i \leq n$ に対する $\mathbf{x}_i$ は何らかのソース分布から抽出され、すべての $1 \leq i \leq m$ に対して $\mathbf{u}_i$ が抽出されると仮定します。ターゲット分布から抽出されます。以下は、共変量シフトを修正するためのプロトタイプのアルゴリズムです。
1. バイナリ分類トレーニング セットを生成します: ${(\mathbf{x}_1, -1), \ldots, (\mathbf{x}_n, -1), (\mathbf{u}_1, 1), \ldots , (\mathbf{u}_m, 1)}$.
1. ロジスティック回帰を使用してバイナリ分類器を学習させ、関数 $h$ を取得します。
1.  $\beta_i = \exp(h(\mathbf{x}_i))$ またはそれ以上の $\beta_i = \min(\exp(h(\mathbf{x}_i)), c)$ を使用してトレーニング データを重み付けします。定数 $c$。
1.  `eq_weighted-empirical-risk-min`の ${(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)}$ のトレーニングには重み $\beta_i$ を使用します。

上記のアルゴリズムは重要な前提に依存していることに注意してください。このスキームが機能するには、ターゲット (テスト時間など) 分布内の各データ例がトレーニング時に発生する確率がゼロ以外である必要があります。 $p(\mathbf{x}) &gt; 0$ であるが $q(\mathbf{x}) = 0$ である点を見つけた場合、対応する重要度の重みは無限大になるはずです。



### ラベルずれ補正

$k$ カテゴリを含む分類タスクを扱っていると仮定します。 :numref: `subsec_covariate-shift-correction`で同じ表記を使用すると、 $q$ と $p$ はそれぞれソース分布 (トレーニング時間など) とターゲット分布 (テスト時間など) になります。ラベルの分布が時間の経過とともに変化すると仮定します: $q(y) \neq p(y)$ ですが、クラス条件付き分布は同じままです: $q(\mathbf{x} \mid y)=p(\mathbf {x} \mid y)$。ソース分布 $q(y)$ が「間違っている」場合は、 :eqref: `eq_true-risk`で定義されているリスクの次のアイデンティティに従ってそれを修正できます。

 $$ \begin{aligned} \int\int l(f(\mathbf{x}), y) p(\mathbf{x} \mid y)p(y) ;d\mathbf{x}dy = \int \int l(f(\mathbf{x}), y) q(\mathbf{x} \mid y)q(y)\frac{p(y)}{q(y)} ;d\mathbf{x }ディ。 \end{整列} $$

ここで、重要度の重みはラベル尤度比に対応します。

 $$\beta_i \stackrel{\mathrm{def}}{=} \frac{p(y_i)}{q(y_i)}.$$

ラベル シフトの優れた点の 1 つは、ソース分布に関して適切なモデルがあれば、アンビエント次元をまったく扱うことなく、これらの重みの一貫した推定値を取得できることです。深層学習では、入力は画像などの高次元オブジェクトになる傾向がありますが、ラベルはカテゴリなどのより単純なオブジェクトであることがよくあります。

ターゲットのラベル分布を推定するには、まず適度に優れた既製の分類器 (通常はトレーニング データでトレーニングされた) を取得し、検証セット (これもトレーニング分布から) を使用してその混同行列を計算します。*混同行列*$\mathbf{C}$ は単なる $k \times k$ 行列で、各列はラベル カテゴリ (グラウンド トゥルース) に対応し、各行はモデルの予測カテゴリに対応します。各セルの値 $c_{ij}$ は、真のラベルが $j$ であり、モデルが $i$ を予測した検証セットの予測合計の割合です。

現在、複雑なリアルタイム アノテーション パイプラインに投資しない限り、実際に目にするサンプルのラベルを見ることができないため、ターゲット データの混同行列を直接計算することはできません。ただし、私たちにできることは、テスト時にすべてのモデルの予測を平均して、平均モデル出力 $\mu(\hat{\mathbf{y}}) \in \mathbb{R}^k$ を求めることです。 $i^\mathrm{th}$ element $\mu(\hat{y}_i)$ は、モデルが $i$ を予測したテスト セットの予測総数の割合です。

いくつかの穏やかな条件下では、分類器がそもそもかなり正確で、ターゲット データに以前に見たカテゴリのみが含まれており、ラベル シフトの仮定が最初に成立する場合 (最も強力な場合) であることがわかります。ここでは仮定)、単純な線形システムを解くことによってテスト セットのラベル分布を推定できます。

 $$\mathbf{C} p(\mathbf{y}) = \mu(\hat{\mathbf{y}}),$$

なぜなら、推定 $\sum_{j=1}^k c_{ij} p(y_j) = \mu(\hat{y}_i)$ がすべての $1 \leq i \leq k$ に対して成り立つからです。ここで $p( y_j)$ は、$k$ 次元のラベル分布ベクトル $p(\mathbf{y})$ の $j^\mathrm{th}$ 要素です。そもそも分類器が十分に正確であれば、混同行列 $\mathbf{C}$ は可逆となり、解 $p(\mathbf{y}) = \mathbf{C}^{-1} が得られます。 \mu(\hat{\mathbf{y}})$。

ソース データのラベルを観察するので、分布 $q(y)$ を推定するのは簡単です。次に、ラベル $y_i$ を持つトレーニング例 $i$ について、推定 $p(y_i)/q(y_i)$ の比率を取得して重み $\beta_i$ を計算し、これを加重経験的リスク最小化に組み込むことができます。 :eqref: `eq_weighted-empirical-risk-min`にあります。

### コンセプトシフト修正

コンセプトの変化を原則に基づいて修正するのははるかに困難です。たとえば、問題が猫と犬の区別から、白と黒の動物の区別の問題に突然変化した状況では、単に新しいラベルを集めてゼロからトレーニングするよりもはるかに優れたことができると考えるのは不合理でしょう。幸いなことに、実際にはそのような極端な変化はまれです。その代わりに、通常はタスクがゆっくりと変化し続けることが起こります。物事をより具体的にするために、いくつかの例を示します。
- コンピュテーショナル広告では、新製品が発売されると、古い製品の人気は下がります。これは、広告の分布とその人気が徐々に変化することを意味し、クリックスルー率の予測変数もそれに合わせて徐々に変化させる必要があります。
- 交通カメラのレンズは環境による磨耗により徐々に劣化し、画質に徐々に影響を及ぼします。
- ニュースの内容は徐々に変化します (つまり、ニュースの大部分は変更されませんが、新しい記事が表示されます)。

このような場合、ネットワークのトレーニングに使用したのと同じアプローチを使用して、ネットワークをデータの変化に適応させることができます。言い換えれば、既存のネットワークの重みを使用し、最初からトレーニングするのではなく、新しいデータでいくつかの更新ステップを実行するだけです。



## 学習上の問題の分類

分布の変化に対処する方法についての知識があれば、機械学習の問題定式化の他の側面をいくつか検討できるようになります。

### バッチ学習

*バッチ学習*では、モデル $f を訓練するために使用する訓練特徴とラベル ${(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)}$ にアクセスできます。 (\mathbf{x})$。その後、このモデルをデプロイして、同じ分布から抽出された新しいデータ $(\mathbf{x}, y)$ をスコア付けします。これは、ここで説明する問題のデフォルトの仮定です。たとえば、猫と犬のたくさんの写真に基づいて猫検出器をトレーニングするかもしれません。トレーニングが完了すると、猫だけを入れるスマート キャットドア コンピューター ビジョン システムの一部として出荷されます。これはその後、顧客の家に設置され、(極端な状況を除いて) 再度更新されることはありません。

### オンライン学習

ここで、データ $(\mathbf{x}_i, y_i)$ が一度に 1 サンプルずつ到着すると想像してください。より具体的には、最初に $\mathbf{x}_i$ を観察し、次に $f(\mathbf{x}_i)$ の推定値を計算する必要があり、それが完了した後にのみ $y_i$ を観察すると仮定します。そしてそれによって、私たちは自分の決定に応じて報酬を受け取るか損失を被ることになります。実際の問題の多くはこのカテゴリに分類されます。たとえば、明日の株価を予測する必要があります。これにより、その予測に基づいて取引できるようになり、一日の終わりには、その予測によって利益が得られるかどうかがわかります。言い換えれば、*オンライン学習*では、新しい観察を踏まえてモデルを継続的に改善する次のサイクルがあります。

 $$ \mathrm{モデル} ~ f_t \longrightarrow \mathrm{データ} ~ \mathbf{x} *t \longrightarrow \mathrm{推定} ~ f_t(\mathbf{x}_t) \longrightarrow\ \mathrm{観測} ~ y_t \longrightarrow \mathrm{loss} ~ l(y_t, f_t(\mathbf{x}_t)) \longrightarrow \mathrm{model} ~ f* {t+1} $$

### 山賊

*盗賊は*上記の問題の特殊なケースです。ほとんどの学習問題では、そのパラメーター (ディープ ネットワークなど) を学習したい、連続的にパラメーター化された関数 $f$ がありますが、*バンディット*問題では、引っ張ることができるアームの数は有限です。私たちが実行できるアクションの数。この単純な問題に対して、最適性の点でより強力な理論的保証が得られることは、それほど驚くべきことではありません。この問題は、あたかも別個の学習環境であるかのように (混乱して) 扱われることが多いため、主にこの問題を列挙します。

### コントロール

多くの場合、環境は私たちがやったことを覚えています。必ずしも敵対的な態度でいるわけではありませんが、記憶しているだけであり、反応は以前に何が起こったかによって異なります。たとえば、コーヒーボイラーコントローラーは、以前にボイラーを加熱していたかどうかに応じて、異なる温度を観察します。そこでは、PID (比例積分微分) コントローラー アルゴリズムが一般的な選択肢です。同様に、ニュース サイトでのユーザーの行動は、以前にユーザーに示した内容に依存します (たとえば、ユーザーはほとんどのニュースを 1 回しか読みません)。このようなアルゴリズムの多くは、アルゴリズムが動作する環境のモデルを形成して、決定がランダムに見えないようにするなどします。最近では、制御理論 (PID バリアントなど) を使用して、ハイパーパラメータを自動的に調整して、より良い解絡と再構成の品質を達成し、生成されたテキストの多様性と生成された画像の再構成の品質を向上させています。:cite: `Shao.Yao.Sun.ea.2020` · 。

### 強化学習

メモリのある環境のより一般的なケースでは、環境が私たちと協力しようとする状況 (協力ゲーム、特に非ゼロサム ゲームの場合) や、環境が勝とうとする状況に遭遇することがあります。チェス、囲碁、バックギャモン、またはスタークラフトは、*強化学習*の例の一部です。同様に、自動運転車用の優れたコントローラーを構築したいと思うかもしれません。他の車は、自動運転車の運転スタイルに自明ではない方法で反応する可能性が高く、たとえば、自動運転車を避けようとしたり、事故を起こそうとしたり、協力しようとしたりします。

### 環境への配慮

上記のさまざまな状況間の重要な違いの 1 つは、静止環境の場合には機能した可能性がある同じ戦略が、環境が適応できる場合には機能しない可能性があることです。たとえば、トレーダーが発見した裁定取引の機会は、それを活用し始めると消滅する可能性が高くなります。環境が変化する速度と方法によって、実行できるアルゴリズムの種類が大きく決まります。たとえば、物事がゆっくりとしか変化しない可能性があることがわかっている場合、推定値もゆっくりとしか変化しないように強制することもできます。環境が瞬間的に変化する可能性はあるが、非常にまれにしか変化しないことがわかっていれば、それを考慮に入れることができます。これらの種類の知識は、意欲的なデータ サイエンティストにとって、概念の変化、つまり解決しようとしている問題が時間の経過とともに変化する場合に対処するために非常に重要です。



## 機械学習における公平性、説明責任、透明性

最後に、機械学習システムを導入するときは、単に予測モデルを最適化するだけではなく、通常は意思決定を (部分的または完全に) 自動化するために使用されるツールを提供することになることに留意することが重要です。これらの技術システムは、結果として生じる決定の対象となる個人の生活に影響を与える可能性があります。予測の検討から決定への飛躍は、新たな技術的な問題だけでなく、慎重に検討する必要がある多くの倫理的な問題も引き起こします。医療診断システムを導入する場合、どの集団にそれが機能する可能性があり、どの集団に機能しない可能性があるかを知る必要があります。部分集団の福祉に対する予見可能なリスクを見落とすと、劣悪なケアを施すことになる可能性があります。さらに、意思決定システムについて熟考したら、一歩下がってテクノロジーをどのように評価するかを再考する必要があります。この範囲の変更の他の影響の中でも、*精度が*正しい尺度になることはほとんどないことがわかります。たとえば、予測をアクションに変換する場合、さまざまな方法での間違いによる潜在的なコスト感度を考慮する必要が生じることがよくあります。画像を誤って分類する 1 つの方法が人種的な手口とみなされる可能性がある一方で、別のカテゴリへの誤分類は無害である場合、意思決定プロトコルを設計する際に社会的価値観を考慮して、それに応じてしきい値を調整する必要があるかもしれません。また、予測システムがどのようにフィードバック ループを引き起こす可能性があるかにも注意したいと考えています。たとえば、犯罪が予測される地域にパトロール警察官を割り当てる予測警察システムについて考えてみましょう。憂慮すべきパターンがどのようにして現れるかは簡単にわかります。
1. 犯罪が多い地域ではパトロールの数も増えます。
1. その結果、これらの地域ではさらに多くの犯罪が発見され、将来の反復で使用できるトレーニング データに組み込まれます。
1. より多くのポジティブな情報にさらされると、モデルはこれらの地域でさらに多くの犯罪が発生すると予測します。
1. 次の反復では、更新されたモデルは同じ地域をさらに重点的にターゲットにし、さらに多くの犯罪が発見されるなどの結果が得られます。

多くの場合、モデルの予測がそのトレーニング データに結び付けられるさまざまなメカニズムは、モデリング プロセスでは考慮されません。これは、研究者が*暴走フィードバック ループ*と呼ぶものにつながる可能性があります。さらに、そもそも正しい問題に取り組んでいるかどうかにも注意したいと考えています。予測アルゴリズムは現在、情報の拡散を仲介する上で大きな役割を果たしています。個人が遭遇したニュースは、その人が*「いいね！」*した一連の Facebook ページによって判断されるべきでしょうか?これらは、機械学習のキャリアで遭遇する可能性のある多くの差し迫った倫理的ジレンマのうちのほんの一部です。

## まとめ

多くの場合、トレーニング セットとテスト セットは同じディストリビューションから取得されたものではありません。これを分布シフトといいます。リスクは、実際の分布から抽出されたデータの母集団全体にわたる損失が予想されることです。ただし、通常、この母集団全体を利用することはできません。経験的リスクは、リスクを近似するためのトレーニング データの平均損失です。実際には、経験に基づいてリスクを最小化します。

対応する仮定の下では、テスト時に共変量とラベルのシフトを検出して修正できます。このバイアスを考慮に入れないと、テスト時に問題が発生する可能性があります。場合によっては、環境が自動化されたアクションを記憶し、驚くべき方法で反応することがあります。私たちはモデルを構築する際にこの可能性を考慮し、モデルと環境が予期せぬ形で絡み合う可能性を考慮して、ライブ システムを監視し続ける必要があります。

## 演習
1. 検索エンジンの動作を変えると何が起こるでしょうか?ユーザーは何をすればよいでしょうか?広告主はどうでしょうか？
1. 共変量シフト検出器を実装します。ヒント: 分類器を構築します。
1. 共変量シフト補正器を実装します。
1. 分布の変化のほかに、経験的リスクがどのようにリスクを近似するかに影響を与える可能性のあるものは何でしょうか?

[ディスカッション](https://discuss.d2l.ai/t/105)
